# ANALYSIS OF MTOR

This notebook contains an analysis of the mTOR system described in Varusai and Nguyen, 2018.

We consider a model of the signaling network for mTOR, the mechanistic Target of Rapamycin (mTOR), an evolutionarily conserved network that controls key cellular processes, especially cell growth and metabolism. The signalling cascade is activated by a variety of receptor tyrosine kinases, most notably the Insulin Receptor.

<img src="images/mTOR-System.png"
     alt="Markdown Monster icon" 
     width="500" height="600"
     style="float: left; margin-right: 10px;" />

In [1]:
import controlSBML as ctl
from controlSBML.siso_closed_loop_designer import SISOClosedLoopDesigner

import control
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
ctl.__version__

'1.0.11'

In [2]:
FIGSIZE = (5, 5)
TIMES = np.linspace(0, 2000, 20000)

# Problem Statement

The biological problem we address is controlling the concentration of phosphorylated mTORC1 through phosphorylated insulin. \ccite{varusai_dynamic_2018} describes this in terms of the species pIRS. The system we study has the single input $pIRS$ and the output mTORC1. It turns out that the mTOR system is bistable with respect to the insulin. Thus, our control objective is to regulate the outputs in the high insulin region where V1 $> 4$.
* Outputs: pmTORC1, pmTORC2
* Inputs: V1
* Control problem: regulate pmTORC1, pmTORC2

## mTOR Model

In [3]:
URL = "https://www.ebi.ac.uk/biomodels/model/download/BIOMD0000000823.2?filename=Varusai2018.xml"
INPUT_NAME = "IR"
INPUT_NAME = "pIRS"
OUTPUT_NAME = "pmTORC1"
SYSTEM = ctl.SBMLSystem(URL, input_names=[INPUT_NAME], output_names=[OUTPUT_NAME],
                       is_fixed_input_species=True)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
ts = SYSTEM.plotModel(figsize=(5,5), end_time=2000, markers=np.repeat("", 20))
for column in ts.columns:
    if not column in [INPUT_NAME, OUTPUT_NAME]:
        del ts[column]
times = ts.index/1000
plt.plot(times, ts[OUTPUT_NAME], color="green")
ax = plt.gca()
ax2 = ax.twinx()
ax2.plot(times, ts[INPUT_NAME], color="red")
ax.set_ylabel(OUTPUT_NAME, color="green")
ax2.set_ylabel(INPUT_NAME, color="red")
ax.set_xlabel("time")

In [ ]:
# pIRS
STAIRCASE = ctl.Staircase(initial_value=1, final_value=3, num_step=5)
BUILDER = ctl.SISOTransferFunctionBuilder(SYSTEM)
response_ts, _ = BUILDER.makeStaircaseResponse(staircase=STAIRCASE, end_time=5000, points_per_time=10)
BUILDER.plotStaircaseResponse(response_ts, figsize=FIGSIZE)

* The system is controllable in the region V1 $>4$, although the control decreases with larger V1 for both complexes.
* The operating region for V1 implies an operating region for pmTORC1 of 180 to 220 and for pmTORC2 0.7 to 1.0.

# System Identification

In [ ]:
FITTER_RESULT = BUILDER.fitTransferFunction(num_numerator=3, num_denominator=4, staircase=STAIRCASE, 
                                            fit_start_time=1000, start_time=0, end_time=5000)
BUILDER.plotFitterResult(FITTER_RESULT, figsize=FIGSIZE)

In [ ]:
TRANSFER_FUNCTION = FITTER_RESULT.transfer_function

In [ ]:
TRANSFER_FUNCTION.poles(), TRANSFER_FUNCTION.zeros()

# Testbed Construction

Here, we build a closed loop system where the open loop is the SBML model of the simulated system. Our has a PI controller.

In [ ]:
setpoint = 10
ts, builder = SYSTEM.simulateSISOClosedLoop(kp=0.1, times=TIMES, is_steady_state=False, 
                                            setpoint=setpoint, num_point=1000, sign=-1,
                                            )
SYSTEM.plotSISOClosedLoop(ts, setpoint, figsize=FIGSIZE, markers=[""])

We are able to simulate the closed loop system, but it's not easy to find design parameters such that: (a) settling times are short; (b) there is not overshoot; and (c) all concentrations are non-negative.

**Notes**
1. Need to use positive feedback. Not clear from the controlability curves.
2. Introducing even a little I behavior makes the controller worse.
3. There is a multimodal behavior of the network that radically changes the long term steady state values. Could this be analyzed by decomposing the system to have multiple tandem controllers. The first controls the effect of IR on pIRS; the second controls the effect of pIRS on pmTORC1.

# Design and Evaluation

In [ ]:
times = np.linspace(0, 10000, 100000)
DESIGNER = SISOClosedLoopDesigner(SYSTEM, TRANSFER_FUNCTION, setpoint=50, times=times)
DESIGNER.design(kp=True, ki=True, kf=True, num_restart=5, max_value=5, max_iteration=50)
_ = DESIGNER.evaluate(figsize=FIGSIZE)

In [ ]:
times = np.linspace(0, 10000, 100000)
new_designer = SISOClosedLoopDesigner(SYSTEM, TRANSFER_FUNCTION, setpoint=20, times=times)
new_designer.set(**DESIGNER.get())
_ = new_designer.evaluate(figsize=FIGSIZE)

In [ ]:
DESIGNER = SISOClosedLoopDesigner(SYSTEM, TRANSFER_FUNCTION, setpoint=10, times=TIMES)
DESIGNER.design(kp=True, num_restart=4, min_value=0.0001, max_value=10)
DESIGNER.evaluate(figsize=FIGSIZE)

In [ ]:
DESIGNER = SISOClosedLoopDesigner(SYSTEM, TRANSFER_FUNCTION, setpoint=2, times=TIMES)
DESIGNER.set(kp=9.999)
DESIGNER.evaluate(figsize=FIGSIZE)